In [1]:
import requests
from bs4 import BeautifulSoup

In [14]:
url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month=11&day=5&year=2019'
page = requests.get(url)


In [64]:
soup.find('td').find('a').text

'Michigan State'

In [60]:
soup = BeautifulSoup(page.content)

for result in soup.find_all('td'):
    print(result)



<td><a href="/cbb/schools/michigan-state/2020.html">Michigan State</a><span class="pollrank"> (1) </span></td>
<td class="right">62</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2019-11-05-21-kentucky.html">Final</a>
</td>
<td><a href="/cbb/schools/kentucky/2020.html">Kentucky</a><span class="pollrank"> (2) </span></td>
<td class="right">69</td>
<td class="right"> 
			</td>
<td><a href="/cbb/schools/kansas/2020.html">Kansas</a><span class="pollrank"> (3) </span></td>
<td class="right">66</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2019-11-05-19-duke.html">Final</a>
</td>
<td><a href="/cbb/schools/duke/2020.html">Duke</a><span class="pollrank"> (4) </span></td>
<td class="right">68</td>
<td class="right"> 
			</td>
<td><a href="/cbb/schools/louisville/2020.html">Louisville</a><span class="pollrank"> (5) </span></td>
<td class="right">87</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2019-11-05-18-miami-fl.html">Final</a>
</td>
<td><a href="/cbb/schools/